env: scGPT

本代码用来在scPerturb数据上跑CellOracle

In [2]:
import scanpy as sc
import pandas as pd
import numpy as np
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

import pickle
import sys
import requests

from types import MethodType
import importlib
from scperturb import *

import anndata as ad

In [3]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns

import celloracle as co
co.__version__


'0.18.0'

In [4]:
# %matplotlib inline

In [5]:
sys.path.append("/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/")

In [6]:
import v1
from v1.utils import *
from v1.dataloader import *

In [7]:
importlib.reload(v1)
importlib.reload(v1.utils)
importlib.reload(v1.dataloader)

<module 'v1.dataloader' from '/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/dataloader.py'>

In [8]:
pd.set_option('display.max_columns', 60)

# 所有gene pert的数据metadata

In [9]:
gene_pert_dict = \
{'AdamsonWeissman2016_GSM2406675_10X001': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 0},
 'AdamsonWeissman2016_GSM2406677_10X005': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 0},
 'AdamsonWeissman2016_GSM2406681_10X010': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 0},
#  'DatlingerBock2017': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 1,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
 'DatlingerBock2021': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'DixitRegev2016': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
#  'FrangiehIzar2021_RNA': {'perturbation_type': 'CRISPR-cas9', # 唯一一个multi celltype，先留着
#   'perturbation_state': 1,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
 
#  'LaraAstiasoHuntly2023_exvivo': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
#  'LaraAstiasoHuntly2023_invivo': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
#  'LaraAstiasoHuntly2023_leukemia': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
#  'LiangWang2023': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 1,
#   'multi_sgRNA': 1},
#  'McFarlandTsherniak2020': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 0},
 'NormanWeissman2019_filtered': {'perturbation_type': 'CRISPRa',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 0},
 'PapalexiSatija2021_eccite_RNA': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'PapalexiSatija2021_eccite_arrayed_RNA': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'ReplogleWeissman2022_K562_essential': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
#  'ReplogleWeissman2022_K562_gwps': {'perturbation_type': 'CRISPRi', # pert太多了，先不测试
#   'perturbation_state': 1,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
 'ReplogleWeissman2022_rpe1': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
#  'SantinhaPlatt2023': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 0},
 'ShifrutMarson2018': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'SunshineHein2023': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'TianKampmann2019_day7neuron': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'TianKampmann2019_iPSC': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'TianKampmann2021_CRISPRa': {'perturbation_type': 'CRISPRa',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'TianKampmann2021_CRISPRi': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'XuCao2023': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1}}


In [10]:
prefix_list = list(gene_pert_dict.keys())
len(prefix_list)

17

# 正式版

## 一次性运行所有dataset

In [16]:

for prefix in prefix_list[:]:
    print('='*20,prefix)

    fix_seed(2024)
    # - init para
    data_dir = '/nfs/public/lichen/data/single_cell/perturb_data/scPerturb/raw/scPerturb_rna/statistic_20240520'
    pert_cell_filter = 100 # this is used to filter perts, cell number less than this will be filtered
    seed = 2024 # this is the random seed
    split_type = 1 # 1 for unseen perts; 0 for unseen celltypes
    split_ratio = [0.7, 0.2, 0.1] # train:test:val; val is used to choose data, test is for final validation
    var_num = 5000 # selecting hvg number
    num_de_genes = 20 # number of de genes
    bs_train = 32 # batch size of trainloader
    bs_test = 32 # batch size of testloader
    lr = 1e-4

    tmp_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/scPerturb'
    # save_prefix = f'GEARS'
    save_prefix = f'GEARS_v2-prefix_{prefix}-pert_cell_filter_{pert_cell_filter}-\
seed_{seed}-split_type_{split_type}-var_num_{var_num}-num_de_genes_{num_de_genes}-bs_train_{bs_train}-\
bs_test_{bs_test}'
    save_dir = os.path.join(tmp_dir, prefix, save_prefix)
    # - load pert_data
    pert_data = pickle.load(open(os.path.join(save_dir,'pert_data.pkl'), 'rb'))

    # - create new dir
    tmp_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark_202410/unseen_perturbation/result/CellOracle'
    save_prefix = f'CellOracle_v1' # v1 version
    save_dir = os.path.join(tmp_dir, prefix, save_prefix)
    
    # - add condition, which is perturbation in gears [TODO] this is only 
    pert_data.adata = pert_data.adata_split
    def get_adata_condition(x):
        if ';' in x['perturbation_new']:
            return '+'.join(x['perturbation_new'].split('; '))
        
        if x['perturbation_new'] == 'control':
            return 'ctrl'
        else:
            return '+'.join([x['perturbation_new'],'ctrl'])
        
    adata_condition = pert_data.adata.obs.apply(get_adata_condition, axis=1)
    pert_data.adata.obs['condition'] = adata_condition
    
    pert_data.adata.uns['rank_genes_groups_cov_all'] = pert_data.adata.uns['rank_genes_groups']
    pert_data.adata.var['gene_name'] = pert_data.adata.var_names
    
    # - condition_name : this variable should be changed if 
    pert_data.adata.obs.loc[:, 'condition_name'] =  pert_data.adata.obs.apply(lambda x: ' | '.join([x.perturbation_new, x.celltype_new]), axis = 1)
    print('add condition finished')

    
    # # - get go genes; special set for GEARS
    # pert_data.get_gene2go()
    # pert_data.set_pert_genes()

    # # - get dataset for gears
    # pert_data.get_Data_gears(num_de_genes = pert_data.num_de_genes,
    #                         dataset_name = ['train', 'test', 'val'])

    # # - add necessary elements for gears
    # pert_data.modify_gears()

    # # - get dataloader
    # trainloader, testloader, valloader = pert_data.get_dataloader(mode='all')
    
    # if len(testloader)==0:
    #     continue
    
    n_cells_downsample = 10000
    threshold_number = 10000
    
    ##########################################################
    
    # - get control adata
    pert_ctrl = ' | '.join(['control',pert_data.filter_perturbation_list[0].split(' | ')[1]])
    adata = pert_data.adata_split[pert_data.adata_split.obs['perturbation_group']==pert_ctrl].copy()
    print(f'adata.shape is: ',adata.shape)

    # -- get the baseGRN
    # Load TF info which was made from mouse cell atlas dataset.
    base_GRN = co.data.load_human_promoter_base_GRN()
    print('base_GRN.shape: ', base_GRN.shape)

    if os.path.exists(os.path.join(save_dir, "ctrl.celloracle.oracle")):
        print('file exists')
        oracle = co.load_hdf5(os.path.join(save_dir, "ctrl.celloracle.oracle"))
        links = co.load_hdf5(file_path=os.path.join(save_dir, "ctrl.celloracle.links"))
        
    else:

        # - start CellOracle process for the whole ctrl

        # -- keep raw cont data before log transformation
        adata.raw = adata
        if not isinstance(adata.raw.X, np.ndarray):
            adata.layers["raw_count"] = (np.exp(adata.raw.X.toarray())-1).copy()
        else:
            adata.layers["raw_count"] = (np.exp(adata.raw.X)-1).copy()
            
        # -- get umap 
        sc.pp.scale(adata)
        # PCA
        sc.tl.pca(adata, svd_solver='arpack', random_state=2022)
        # UMAP
        sc.pp.neighbors(adata, n_neighbors=4, n_pcs=20, random_state=2022)
        sc.tl.umap(adata,random_state=2022)

        # -- Random downsampling into 30K cells if the anndata object include more than 30 K cells.
        if adata.shape[0] > n_cells_downsample:
            # Let's dowmsample into 30K cells
            sc.pp.subsample(adata, n_obs=n_cells_downsample, random_state=123)
        print(f"Cell number is :{adata.shape[0]}")

        # -- Instantiate Oracle object
        oracle = co.Oracle()

        # -- Check data in anndata
        print("Metadata columns :", list(adata.obs.columns))
        print("Dimensional reduction: ", list(adata.obsm.keys()))

        # -- In this notebook, we use the unscaled mRNA count for the nput of Oracle object.
        adata.X = adata.layers["raw_count"].copy()

        # -- Instantiate Oracle object.
        oracle.import_anndata_as_raw_count(adata=adata,
                                        cluster_column_name="celltype",
                                        embedding_name="X_umap")

        # -- You can load TF info dataframe with the following code.
        oracle.import_TF_data(TF_info_matrix=base_GRN)

        # -- knn imputation, this step is needed for the whole ctrl
        # Perform PCA
        oracle.perform_PCA()

        # Select important PCs
        plt.plot(np.cumsum(oracle.pca.explained_variance_ratio_)[:100])
        n_comps = np.where(np.diff(np.diff(np.cumsum(oracle.pca.explained_variance_ratio_))>0.002))[0][0]
        plt.axvline(n_comps, c="k")
        plt.show()
        print(n_comps)
        n_comps = min(n_comps, 50)

        n_cell = oracle.adata.shape[0]
        print(f"cell number is :{n_cell}")

        k = int(0.025*n_cell)
        print(f"Auto-selected k is :{k}")

        oracle.knn_imputation(n_pca_dims=n_comps, k=k, balanced=True, b_sight=k*8,
                            b_maxl=k*4, n_jobs=4)

        # model_prefix = ''
        os.makedirs(save_dir, exist_ok=True)
        # -- save the oracle
        oracle.to_hdf5(os.path.join(save_dir, "ctrl.celloracle.oracle"))

        # -- get the links
        # Calculate GRN for each population in "louvain_annot" clustering unit.
        # This step may take some time.(~30 minutes)
        links = oracle.get_links(cluster_name_for_GRN_unit="celltype", alpha=10,
                                verbose_level=10)

        # -- Save Links object.
        links.to_hdf5(file_path=os.path.join(save_dir, "ctrl.celloracle.links"))


    # -- filter and get the coef_mtx
    links.filter_links(threshold_number=threshold_number,
                        p=0.001,
                        weight='coef_abs')
    oracle.get_cluster_specific_TFdict_from_Links(links_object=links)
    oracle.fit_GRN_for_simulation(alpha=10,
                                use_cluster_specific_TFdict=True)


    ######################################################

    # - get all the TFs in the base_GRN
    TFdict = import_TF_data(TF_info_matrix=base_GRN)
    tf_target_dict = {}
    for target, gene_set in TFdict.items():
        for tf in gene_set:
            if tf not in tf_target_dict:
                tf_target_dict[tf] = []
                tf_target_dict[tf].append(target)
            else:
                tf_target_dict[tf].append(target)
    total_tf_list = list(tf_target_dict.keys())

    common_pert = []
    for perts in pert_data.test_perts:
        flag = True
        for pert in perts.split('; '):
            if pert not in total_tf_list:
                flag = False
                break
        if flag:
            common_pert.append(perts)
    common_pert_2 = []       
    for perts in common_pert:
        flag = True
        for pert in perts.split('; '):
            if pert not in adata.var_names:
                flag = False
                break
        if flag:
            common_pert_2.append(perts)
    print('common_pert num is: ',len(common_pert_2))
    print(common_pert_2)
    if len(common_pert_2) == 0:
        continue
    
    
    ###########################################
    celltype = adata.obs['celltype_new'].unique()[0]
    
    # - get the tf_GRN_dict, to check whether pert have regulatory relations
    gene_GRN_mtx = oracle.coef_matrix_per_cluster[celltype].copy()
    tf_GRN_mtx = gene_GRN_mtx[~(gene_GRN_mtx == 0).all(axis=1)]
    # - get TF-target pair and the regulatory values
    tf_GRN_dict = {} # the tf to targets
    for i in range(len(tf_GRN_mtx)):
        tmp = tf_GRN_mtx.iloc[i,:]
        tmp = tmp[tmp!=0]

        tf_GRN_dict[tf_GRN_mtx.index[i]] = {}
        for j in range(len(tmp)):
            tf_GRN_dict[tf_GRN_mtx.index[i]][tmp.index[j]] = tmp.values[j]
    
    
    pert_cat_list, pred_list, truth_list, pred_de_list, truth_de_list = [], [], [], [], []

    for pert_suffix in common_pert_2:
        pert = ' | '.join([pert_suffix,celltype])
        
        adata_pert = pert_data.adata_split[pert_data.adata_split.obs['perturbation_group']==pert].copy()
        adata_ctrl = pert_data.adata_split[list(adata_pert.obs['control_barcode'])].copy()
        # - this is for crispra
        gois = pert.split(' | ')[0].split('; ')
        goi_dict = {}
        if gene_pert_dict[prefix]['perturbation_type'] == 'CRISPRa':
            for goi in gois:
                # -- if original value is zero
                if np.mean(adata_ctrl[:,goi].X.toarray())==0:
                    print(f'{goi} ctrl expression is 0')
                    continue
                # -- if the TF has no targets
                if goi not in list(tf_GRN_dict.keys()):
                    print(f'{goi} is not in the tf_GRN_dict, no targets')
                    continue
                goi_dict[goi] = np.mean(adata_ctrl[:,goi].X.toarray())+1
        else:
            for goi in gois:
                # -- if original value is zero
                if np.mean(adata_ctrl[:,goi].X.toarray())==0:
                    print(f'{goi} ctrl expression is 0')
                    continue
                # -- if the TF has no targets
                if goi not in list(tf_GRN_dict.keys()):
                    print(f'{goi} is not in the tf_GRN_dict, no targets')
                    continue
                goi_dict[goi] = 0
        if len(goi_dict) == 0:
            print(f'{pert} is filtered')
            continue


        # - get oracle_ctrl

        # keep raw cont data before log transformation
        adata_ctrl.raw = adata_ctrl

        # the result will be recovered in normalized_count
        if not isinstance(adata_ctrl.raw.X, np.ndarray):
            adata_ctrl.layers["raw_count"] = (np.exp(adata_ctrl.raw.X.toarray())-1).copy()
        else:
            adata_ctrl.layers["raw_count"] = (np.exp(adata_ctrl.raw.X)-1).copy()
            
        sc.pp.scale(adata_ctrl)
        # PCA
        sc.tl.pca(adata_ctrl, svd_solver='arpack', random_state=2022)

        # Diffusion map
        sc.pp.neighbors(adata_ctrl, n_neighbors=4, n_pcs=20, random_state=2022)
        sc.tl.umap(adata_ctrl,random_state=2022)

        # Instantiate Oracle object
        oracle_ctrl = co.Oracle()

        # In this notebook, we use the unscaled mRNA count for the nput of Oracle object.
        adata_ctrl.X = adata_ctrl.layers["raw_count"].copy()

        # Instantiate Oracle object.
        oracle_ctrl.import_anndata_as_raw_count(adata=adata_ctrl,
                                        cluster_column_name="celltype",
                                        embedding_name="X_umap")

        # You can load TF info dataframe with the following code.
        oracle_ctrl.import_TF_data(TF_info_matrix=base_GRN)

        # get the imputed_count, here we dont do the impute to get the prediction
        oracle_ctrl.adata.layers["imputed_count"] = oracle_ctrl.adata.layers["normalized_count"].copy()

        # get the coef from the whole ctrl
        oracle_ctrl.coef_matrix_per_cluster = oracle.coef_matrix_per_cluster

        # Enter perturbation conditions to simulate signal propagation after the perturbation.
        oracle_ctrl.simulate_shift(perturb_condition=goi_dict,
                            n_propagation=3)
        # - get the prediction; delta_X = simulated_count - imputed_count
        delta_X, simulated_count = oracle_ctrl.adata.layers["delta_X"], oracle_ctrl.adata.layers["simulated_count"]
        
        # - get test_res
        pert_gears = transform_name(pert)
        geneid2idx = dict(zip(pert_data.adata.var.index.values, range(len(pert_data.adata.var.index.values))))
        pert2pert_full_id = dict(pert_data.adata.obs[['condition', 'condition_name']].values)
        de_idx = [geneid2idx[i] for i in pert_data.adata.uns['rank_genes_groups_cov_all'][pert2pert_full_id[pert_gears]][:20]]
        
        # - add to list
        pert_cat_list.append(np.array([pert_gears]*delta_X.shape[0]))
        pred_list.append(simulated_count)
        truth_list.append(adata_pert.X.toarray())
        pred_de_list.append(simulated_count[:,de_idx])
        truth_de_list.append(adata_pert.X.toarray()[:, de_idx])

    test_res = {}
    test_res['pert_cat'] = np.hstack(pert_cat_list)
    test_res['pred'] = np.vstack(pred_list)
    test_res['truth'] = np.vstack(truth_list)
    test_res['pred_de'] = np.vstack(pred_de_list)
    test_res['truth_de'] = np.vstack(truth_de_list)

    # - get the out
    out = get_metric(pert_data.adata, test_res)
    # out_non_dropout = non_dropout_analysis(pert_data.adata, test_res)


    pickle.dump(out, open(os.path.join(save_dir,f'out.pkl'), 'wb'))
    # pickle.dump(out_non_dropout, open(os.path.join(save_dir,f'out_non_dropout.pkl'), 'wb'))
    
    # break


==================== AdamsonWeissman2016_GSM2406675_10X001
add condition finished
adata.shape is:  (1769, 5001)
Loading prebuilt promoter base-GRN. Version: hg19_gimmemotifsv5_fpr2
base_GRN.shape:  (37003, 1096)
file exists


  0%|          | 0/1 [00:00<?, ?it/s]

common_pert num is:  2
['DDIT3', 'EP300']


... storing 'perturbation' as categorical
... storing 'tissue_type' as categorical
... storing 'cell_line' as categorical
... storing 'disease' as categorical
... storing 'perturbation_type' as categorical
... storing 'celltype' as categorical
... storing 'organism' as categorical
... storing 'perturbation_new' as categorical
... storing 'perturbation_type_new' as categorical
... storing 'celltype_new' as categorical
... storing 'sgRNA_new' as categorical
... storing 'perturbation_group' as categorical
... storing 'data_split' as categorical
... storing 'retain' as categorical
... storing 'control_barcode' as categorical
... storing 'sgRNA_ID' as categorical
... storing 'pert_sgRNA' as categorical
... storing 'condition' as categorical
... storing 'condition_name' as categorical


5001 genes were found in the adata. Note that Celloracle is intended to use around 1000-3000 genes, so the behavior with this number of genes may differ from what is expected.


...

... storing 'perturbation' as categorical
... storing 'guide_id' as categorical
... storing 'Cell_cycle_phase' as categorical
... storing 'target' as categorical
... storing 'perturbation_type' as categorical
... storing 'disease' as categorical
... storing 'tissue_type' as categorical
... storing 'cell_line' as categorical
... storing 'celltype' as categorical
... storing 'organism' as categorical
... storing 'perturbation_new' as categorical
... storing 'perturbation_type_new' as categorical
... storing 'celltype_new' as categorical
... storing 'sgRNA_new' as categorical
... storing 'perturbation_group' as categorical
... storing 'data_split' as categorical
... storing 'retain' as categorical
... storing 'control_barcode' as categorical
... storing 'sgRNA_ID' as categorical
... storing 'pert_sgRNA' as categorical
... storing 'condition' as categorical
... storing 'condition_name' as categorical


5146 genes were found in the adata. Note that Celloracle is intended to use around 1000-3000 genes, so the behavior with this number of genes may differ from what is expected.


... storing 'perturbation' as categorical
... storing 'guide_id' as categorical
... storing 'Cell_cycle_phase' as categorical
... storing 'target' as categorical
... storing 'perturbation_type' as categorical
... storing 'disease' as categorical
... storing 'tissue_type' as categorical
... storing 'cell_line' as categorical
... storing 'celltype' as categorical
... storing 'organism' as categorical
... storing 'perturbation_new' as categorical
... storing 'perturbation_type_new' as categorical
... storing 'celltype_new' as categorical
... storing 'sgRNA_new' as categorical
... storing 'perturbation_group' as categorical
... storing 'data_split' as categorical
... storing 'retain' as categorical
... storing 'control_barcode' as categorical
... storing 'sgRNA_ID' as categorical
... storing 'pert_sgRNA' as categorical
... storing 'condition' as categorical
... storing 'condition_name' as categorical


get metrics... ...


  0%|          | 0/5 [00:00<?, ?it/s]/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/utils.py:720: RuntimeWarning: invalid value encountered in divide
  change_ratio = np.abs(pred-truth)/truth
/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/utils.py:720: RuntimeWarning: invalid value encountered in divide
  change_ratio = np.abs(pred-truth)/truth
/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/utils.py:720: RuntimeWarning: invalid value encountered in divide
  change_ratio = np.abs(pred-truth)/truth
/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/utils.py:720: RuntimeWarning: invalid value encountered in divide
  change_ratio = np.abs(pred-truth)/truth
/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/utils.py:720: RuntimeWarning: invalid value encountered in divide
  change_ratio = np.abs(pred-truth)/truth
/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert

In [13]:
pert_data.modify_gears(without_subgroup = False)

add adata finished
add condition finished


AttributeError: 'Byte_Pert_Data' object has no attribute 'dataset_perturbation_list'

In [47]:
out

{'DDIT3+ctrl': {'frac_correct_direction_all': 0.5490901819636073,
  'frac_correct_direction_top20_de': 0.2,
  'frac_correct_direction_top50_de': 0.12,
  'frac_correct_direction_top100_de': 0.12,
  'frac_correct_direction_top200_de': 0.15,
  'frac_correct_direction_all_de': 0.5490901819636073,
  'frac_correct_direction_20_nonzero': 0.2,
  'frac_in_range': 1.0,
  'frac_in_range_45_55': 0.35,
  'frac_in_range_40_60': 0.5,
  'frac_in_range_25_75': 1.0,
  'mean_sigma': 0.24109451336757576,
  'std_sigma': 0.07616755175733825,
  'frac_sigma_below_1': 1.0,
  'frac_sigma_below_2': 1.0,
  'pearson_delta': 0.01756796584103145,
  'pearson_delta_de': -0.028187475489290216,
  'fold_change_gap_all': 0.038811493479702974,
  'pearson_delta_top200_hvg': 0.009680739307155005,
  'pearson_top200_hvg': 0.9977266465165873,
  'mse_top200_hvg': 0.0034803336259919947,
  'pearson_delta_top20_de': -0.028187475489290216,
  'pearson_top20_de': 0.9956921503915804,
  'mse_top20_de': 0.01583192508479619,
  'pearson_de

In [12]:
prefix, pert

('DatlingerBock2021', 'NR4A1 | T cells')

In [32]:
goi not in list(tf_GRN_dict.keys())

True

In [28]:
prefix_list.index(prefix)

3

In [17]:
len(tf_target_dict['NR4A1'])

4553

In [42]:
np.mean(adata_ctrl[:,'AHR'].X.toarray()), np.mean(adata_ctrl[:,'KLF1'].X.toarray())

(0.0, 0.8494193)

In [27]:
len(list(total_tf_GRN_dict[cluster].keys()))

151

In [12]:
prefix_list.index('TianKampmann2019_day7neuron')

12

In [31]:
pert_cat_list, pred_list, truth_list, pred_de_list, truth_de_list = [], [], [], [], []

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.78645015],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.61073571],
       [0.        , 0.        , 0.        , ..., 0.        , 0.51516783,
        0.51516783]])

In [37]:
out_non_dropout = non_dropout_analysis(pert_data.adata, test_res)